In [1]:
from bs4 import BeautifulSoup                        
import requests
import json
import base64
import pandas as pd
import os

# Dùng cho history data

## VCB

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

keyspace = 'bank_exrate'
username = 'cassandra'
password = 'cassandra'

# Replace with your cluster's contact points
contact_points = ['localhost']
# Replace with your cluster's port (default is 9042)
port = 9042

# Optional: If using authentication
auth_provider = PlainTextAuthProvider(username, password)

# Connect to the cluster
cluster = Cluster(contact_points, port=port, auth_provider=auth_provider)
session = cluster.connect(keyspace)

# Execute a query
query = "SELECT * FROM exrate;"
rows = session.execute(query)

# Process the results
for row in rows:
    print(row)
    
# Clean up and close the connection
session.shutdown()
cluster.shutdown()

In [ ]:
def url_routing(bank: str, date: str):
    if (bank.lower == 'vietcombank'):
        return f"https://www.vietcombank.com.vn/api/exchangerates?date={date}" # e.g. 2024-06-07

    return ""

In [11]:
dataStr = requests.get(f"https://www.vietcombank.com.vn/api/exchangerates?date=2024-06-07").content
dataMap = eval(dataStr)

data = dataMap['Data']
data
dataMap

{'Count': 20,
 'Date': '2024-06-07T00:00:00',
 'UpdatedDate': '2024-06-07T23:00:00+07:00',
 'Data': [{'currencyName': 'US DOLLAR',
   'currencyCode': 'USD',
   'cash': '25183.00',
   'transfer': '25213.00',
   'sell': '25453.00',
   'icon': '/-/media/Default-Website/Default-Images/Icons/Flags/im_flag_usa.svg?h=32&w=32&ts=20230610195953&hash=4576D9B7C910424CC72B7A592BA369AE'},
  {'currencyName': 'EURO',
   'currencyCode': 'EUR',
   'cash': '26973.23',
   'transfer': '27245.69',
   'sell': '28452.17',
   'icon': '/-/media/Default-Website/Default-Images/Icons/Flags/im_flag_eur.svg?h=32&w=32&ts=20230610195955&hash=10502813C2316A5601FD57F7A0C89A3A'},
  {'currencyName': 'UK POUND STERLING',
   'currencyCode': 'GBP',
   'cash': '31676.83',
   'transfer': '31996.80',
   'sell': '33023.24',
   'icon': '/-/media/Default-Website/Default-Images/Icons/Flags/im_flag_gbp.svg?h=32&w=32&ts=20230610195950&hash=72ED63A02E20F8B413E65DE9B7F062AE'},
  {'currencyName': 'JAPANESE YEN',
   'currencyCode': 'JPY

In [16]:
import time

int(time.time() * 1000)

1719048126485

In [55]:
list_columns = ['USD', 'AUD', 'CAD', 'CHF', 'EUR', 'JPY', 'SGD']
dfVcb = pd.DataFrame(data)[['currencyCode', 'cash', 'transfer', 'sell']] 
dfVcb = dfVcb.loc[dfVcb['currencyCode'].isin(list_columns)]
dfVcb = dfVcb.astype({'cash': 'float', 'transfer': 'float', 'sell': 'float'})

dfVcb

,currencyCode,cash,transfer,sell
0,USD,25183.00,25213.00,25453.00
1,EUR,26973.23,27245.69,28452.17
3,JPY,158.29,159.88,167.53
4,AUD,16510.89,16677.67,17212.68
5,SGD,18414.14,18600.14,19196.83
7,CAD,18129.81,18312.94,18900.41
8,CHF,27836.39,28117.56,29019.56


In [65]:
df_pivot = None

for item in ['cash', 'transfer', 'sell']:
    dfVcb2 = dfVcb[['currencyCode', item]].T
    dfVcb2.columns = dfVcb2.iloc[0]
    dfVcb2 = dfVcb2.drop(dfVcb2.index[0])
    
    if df_pivot is None:
        df_pivot = dfVcb2
    else:
        df_pivot = pd.concat([df_pivot, dfVcb2], axis=0)

# df_pivot = pd.concat([df_pivot, df_pivot.iloc[[-1]]], axis=0)
df_pivot

currencyCode,USD,EUR,JPY,AUD,SGD,CAD,CHF
cash,25183.0,26973.23,158.29,16510.89,18414.14,18129.81,27836.39
transfer,25213.0,27245.69,159.88,16677.67,18600.14,18312.94,28117.56
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56


In [68]:
df_pivot = df_pivot.assign(
    instrument_type=df_pivot.index.map(lambda x: 'buy' if x != 'sell' else x),
    deal_type = df_pivot.index.map(lambda x: 'cash' if x == 'sell' else x)
)
df_pivot

currencyCode,USD,EUR,JPY,AUD,SGD,CAD,CHF,instrument_type,deal_type
cash,25183.0,26973.23,158.29,16510.89,18414.14,18129.81,27836.39,buy,cash
transfer,25213.0,27245.69,159.88,16677.67,18600.14,18312.94,28117.56,buy,transfer
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56,sell,cash


In [76]:
df_pivot = pd.concat([df_pivot, df_pivot.iloc[[-1], :-1].assign(deal_type=['transfer'])], axis=0)
df_pivot

currencyCode,USD,EUR,JPY,AUD,SGD,CAD,CHF,instrument_type,deal_type
cash,25183.0,26973.23,158.29,16510.89,18414.14,18129.81,27836.39,buy,cash
transfer,25213.0,27245.69,159.88,16677.67,18600.14,18312.94,28117.56,buy,transfer
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56,sell,cash
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56,sell,transfer


In [78]:
df_pivot = df_pivot.assign(bank='vietcombank')
df_pivot

currencyCode,USD,EUR,JPY,AUD,SGD,CAD,CHF,instrument_type,deal_type,bank
cash,25183.0,26973.23,158.29,16510.89,18414.14,18129.81,27836.39,buy,cash,vietcombank
transfer,25213.0,27245.69,159.88,16677.67,18600.14,18312.94,28117.56,buy,transfer,vietcombank
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56,sell,cash,vietcombank
sell,25453.0,28452.17,167.53,17212.68,19196.83,18900.41,29019.56,sell,transfer,vietcombank


In [80]:
df_pivot.to_csv('test.csv', index=False)

In [79]:
for index, row in df_pivot.iterrows():
    print(index)
    print(row)

cash
currencyCode
USD                    25183.0
EUR                   26973.23
JPY                     158.29
AUD                   16510.89
SGD                   18414.14
CAD                   18129.81
CHF                   27836.39
instrument_type            buy
deal_type                 cash
bank               vietcombank
Name: cash, dtype: object
transfer
currencyCode
USD                    25213.0
EUR                   27245.69
JPY                     159.88
AUD                   16677.67
SGD                   18600.14
CAD                   18312.94
CHF                   28117.56
instrument_type            buy
deal_type             transfer
bank               vietcombank
Name: transfer, dtype: object
sell
currencyCode
USD                    25453.0
EUR                   28452.17
JPY                     167.53
AUD                   17212.68
SGD                   19196.83
CAD                   18900.41
CHF                   29019.56
instrument_type           sell
deal_type         

In [ ]:
# TODO: 
#   add bank_name as param
#   try-catch
def send_request(date: str) -> str:
    data = requests.get(f"https://www.vietcombank.com.vn/api/exchangerates/exportexcel?date={date}").content
    response_dict = json.loads(data)
    file_name = response_dict['FileName'].replace(" ", "_")
    base64_data = response_dict['Data']

    # decode base64 data into byte string
    file_data = base64.b64decode(base64_data)

    # export byte string to file
    with open(file_name, 'wb') as file:
        file.write(file_data)
    
    return file_name

def get_bank_data(date: str):
    exrate_file_path = send_request(date)
    
    excel_df = pd.read_excel(exrate_file_path, skiprows=3, header=None) \
        .rename(columns={0: "currency_code", 1: "currency_name", 2: "buy_rate_cash", 3: "buy_rate_transfer", 4: "sell_rate"})

    # remove metadata files
    excel_df = excel_df.loc[(excel_df['currency_code'].isna() == False) & (excel_df['currency_code'].str.len() == 3)]
    
    # remove file after transform
    if os.path.exists(exrate_file_path):
        os.remove(exrate_file_path)
        
    return excel_df

In [ ]:
df = get_bank_data('2024-06-06')
df.head(100)

## Techcombank

In [1]:
import requests

url = 'https://techcombank.com/content/techcombank/web/vn/vi/cong-cu-tien-ich/ty-gia/_jcr_content.exchange-rates.2024-07-01.integration.json'
response_str = requests.get(url).content
response_str

# map_response = eval(response_str)
# map_data = map_response['exchangeRate']['data']

b'{"exchangeRate":{"data":[{"itemId":"11272","label":"AUD","askRate":"17361.0","bidRateCK":"16732.0","bidRateTM":"16463.0","sourceCurrency":"AUD","targetCurrency":"VND","askRateTM":"17372.0"},{"itemId":"10295","label":"CAD","askRate":"18962.0","bidRateCK":"18342.0","bidRateTM":"18067.0","sourceCurrency":"CAD","targetCurrency":"VND","askRateTM":"18969.0"},{"itemId":"10524","label":"CHF","askRate":"28614.0","bidRateCK":"27981.0","bidRateTM":"27617.0","sourceCurrency":"CHF","targetCurrency":"VND","askRateTM":"28627.0"},{"itemId":"11811","label":"CNY","askRate":"3710.0","bidRateCK":"3320.0","sourceCurrency":"CNY","targetCurrency":"VND"},{"itemId":"10262","label":"EUR","askRate":"27840.0","bidRateCK":"27011.0","bidRateTM":"26752.0","sourceCurrency":"EUR","targetCurrency":"VND","askRateTM":"28104.0"},{"itemId":"10106","label":"GBP","askRate":"32767.0","bidRateCK":"31831.0","bidRateTM":"31451.0","sourceCurrency":"GBP","targetCurrency":"VND","askRateTM":"32786.0"},{"itemId":"11729","label":"HK

In [12]:
from datetime import datetime

map_response = eval(response_str)

now = datetime.now().strftime("%Y-%m-%d") + " " + map_response['exchangeRate']['updatedTimes'][0]
pd.Timestamp(now)

# list_map_data = map_response['exchangeRate']['data']
# list_map_data

Timestamp('2024-07-03 17:02:00')

In [11]:
list_map_data[0].keys()

dict_keys(['itemId', 'label', 'askRate', 'bidRateCK', 'bidRateTM', 'sourceCurrency', 'targetCurrency', 'askRateTM'])

In [4]:
import pandas as pd

map_data_trans = {}
cassandra_mapping = {'bidRateTM': "buy_cash", 'bidRateCK': "buy_transfer", 'askRateTM': "sell_cash", 'askRate': "sell_transfer"}
list_currency = ['USD', 'AUD', 'CAD', 'CHF', 'EUR', 'JPY', 'SGD']
select_columns = ['bank', 'deal_type', 'instrument_type'] + list_currency

for map_data in list_map_data:
    list_map_data_by_item = []
    currency = map_data['label']
    
    for techcombank_key, cassandra_key in cassandra_mapping.items():
        if techcombank_key in map_data:
            deal_type, instrument_type = cassandra_key.split('_')
            list_cassandra_value = map_data_trans.get(cassandra_key, {'deal_type': deal_type, 'instrument_type': instrument_type, 'bank': 'techcombank'})
            list_cassandra_value[currency] = float(map_data[techcombank_key])
            
            map_data_trans[cassandra_key] = list_cassandra_value

list_rows = []
for row in map_data_trans.values():
    list_rows.append(row)
    
df_exrate = pd.DataFrame(list_rows)
df_exrate['USD'] = df_exrate[['USD (1,2)', 'USD (5,10,20)', 'USD (50,100)']].mean(axis=1, skipna=True)
df_exrate = df_exrate[select_columns]
df_exrate

,bank,deal_type,instrument_type,USD,AUD,CAD,CHF,EUR,JPY,SGD
0,techcombank,buy,cash,25210.0,16463.0,18067.0,27617.0,26752.0,151.34,18245.0
1,techcombank,buy,transfer,25273.0,16732.0,18342.0,27981.0,27011.0,155.47,18520.0
2,techcombank,sell,cash,25464.0,17372.0,18969.0,28627.0,28104.0,163.73,19156.0
3,techcombank,sell,transfer,25464.0,17361.0,18962.0,28614.0,27840.0,161.62,19046.0


In [5]:
df_exrate.dtypes

bank                object
deal_type           object
instrument_type     object
USD                float64
AUD                float64
CAD                float64
CHF                float64
EUR                float64
JPY                float64
SGD                float64
dtype: object

In [28]:
data2 = [
    {'id': 1, 'name': 'Alice', 'age': 25},
    {'id': 2, 'name': 'Bob'},
    {'id': 3, 'name': 'Catherine', 'age': 22}
]
df2 = pd.DataFrame(data2)
df2

,id,name,age
0,1,Alice,25.0
1,2,Bob,NaN
2,3,Catherine,22.0


In [26]:
import pandas as pd

data1 = {
    'usd': [1],
    'aud': [2],
    'eur': [3]
}
df1 = pd.DataFrame(data1)
df1

,usd,aud,eur
0,1,2,3


# Time uuid 

In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import datetime

In [2]:
keyspace = 'bank_exrate'
username = 'cassandra'
password = 'cassandra'
contact_points = ['127.0.0.1']
port = 9042
auth_provider = PlainTextAuthProvider(username, password)

cluster = Cluster(contact_points, port=port, auth_provider=auth_provider)
session = cluster.connect(keyspace)

In [9]:
str(cassandra.util.uuid_from_time(datetime.datetime.now()))

'd4fe2962-309d-11ef-a427-39e14a732f44'

In [7]:
datetime.datetime.now().timestamp()

1719633239.508103

In [10]:
# server timezone
datetime.datetime.now()

datetime.datetime(2024, 6, 29, 10, 54, 23, 48428)

In [11]:
# utc
cassandra.util.datetime_from_timestamp(datetime.datetime.now().timestamp())

datetime.datetime(2024, 6, 29, 3, 54, 23, 321149)

# Concurrent processing

In [14]:
import concurrent.futures
import time

def func1():
    try:
        print("Starting func1")
        time.sleep(2)  # Simulate a long-running task
        raise ValueError("An error occurred in func1")
    except Exception as e:
        print(f"Exception in func1: {e}")

def func2():
    try:
        print("Starting func2")
        time.sleep(4)  # Simulate a long-running task
        print("func2 completed successfully")
        
        raise ValueError("An error occurred in func2")
        
    except Exception as e:
        print(f"Exception in func2: {e}")

# Run functions in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(func1)
    future2 = executor.submit(func2)

    # Optionally wait for the results (not blocking)
    concurrent.futures.wait([future1, future2], return_when=concurrent.futures.ALL_COMPLETED)
    print("Both functions have completed")

# Checking for results and handling exceptions if needed
if future1.exception():
    print(f"func1 raised an exception: {future1.exception()}")
if future2.exception():
    print(f"func2 raised an exception: {future2.exception()}")


Starting func1
Starting func2
Exception in func1: An error occurred in func1
func2 completed successfully
Exception in func2: An error occurred in func2
Both functions have completed


In [22]:
from datetime import datetime, timedelta

# Set the start date and end date
start_date_str = '2024-06-27'
end_date_str = '2020-01-01'

# Parse the start and end dates into datetime objects
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

# Loop from start_date to end_date, decrementing by one day
current_date = start_date
while current_date >= end_date:
    # Print the current date (or perform any other operation)
    print(current_date.strftime('%Y-%m-%d'))
    
    # Decrement the current date by one day
    current_date -= timedelta(days=1)


while True:
    

2024-06-27
2024-06-26
2024-06-25
2024-06-24
2024-06-23
2024-06-22
2024-06-21
2024-06-20
2024-06-19
2024-06-18
2024-06-17
2024-06-16
2024-06-15
2024-06-14
2024-06-13
2024-06-12
2024-06-11
2024-06-10
2024-06-09
2024-06-08
2024-06-07
2024-06-06
2024-06-05
2024-06-04
2024-06-03
2024-06-02
2024-06-01
2024-05-31
2024-05-30
2024-05-29
2024-05-28
2024-05-27
2024-05-26
2024-05-25
2024-05-24
2024-05-23
2024-05-22
2024-05-21
2024-05-20
2024-05-19
2024-05-18
2024-05-17
2024-05-16
2024-05-15
2024-05-14
2024-05-13
2024-05-12
2024-05-11
2024-05-10
2024-05-09
2024-05-08
2024-05-07
2024-05-06
2024-05-05
2024-05-04
2024-05-03
2024-05-02
2024-05-01
2024-04-30
2024-04-29
2024-04-28
2024-04-27
2024-04-26
2024-04-25
2024-04-24
2024-04-23
2024-04-22
2024-04-21
2024-04-20
2024-04-19
2024-04-18
2024-04-17
2024-04-16
2024-04-15
2024-04-14
2024-04-13
2024-04-12
2024-04-11
2024-04-10
2024-04-09
2024-04-08
2024-04-07
2024-04-06
2024-04-05
2024-04-04
2024-04-03
2024-04-02
2024-04-01
2024-03-31
2024-03-30
2024-03-29

In [20]:
def convert_utc_to_ho_chi_minh(utc_datetime_str):
    # Define the format of the input datetime string
    utc_format = '%Y-%m-%d %H:%M:%S'
    
    # Parse the datetime string into a datetime object
    utc_time = datetime.strptime(utc_datetime_str, utc_format)
    
    # Set the timezone to UTC
    utc_time = utc_time.replace(tzinfo=pytz.utc)
    
    # Define the Ho Chi Minh timezone
    ho_chi_minh_tz = pytz.timezone('Asia/Ho_Chi_Minh')
    
    # Convert the datetime object to Ho Chi Minh timezone
    ho_chi_minh_time = utc_time.astimezone(ho_chi_minh_tz)
    
    return ho_chi_minh_time

# Example usage
utc_datetime_str = '2024-06-27 07:30:45'
ho_chi_minh_time = convert_utc_to_ho_chi_minh(utc_datetime_str)

# Print the result
print("UTC time:", utc_datetime_str)
print("Ho Chi Minh time:", ho_chi_minh_time.strftime('%Y-%m-%d %H:%M:%S'))

UTC time: 2024-06-27 07:30:45
Ho Chi Minh time: 2024-06-27 14:30:45


In [18]:
utc_time

datetime.datetime(2024, 7, 3, 10, 2)

# Draft

In [51]:
import pandas as pd

# Sample DataFrames
data1 = {
    'bank': ['Bank A', 'Bank B', 'Bank C'],
    'deal_type': ['buy', 'sell', 'buy'],
    'instrument_type': ['cash', 'transfer', 'cash'],
    'last_updated': ['2024-06-07T00:00:00', '2024-06-08T00:00:00', '2024-06-09T00:00:00'],
    'usd': [1.1, 1.2, 1.3],
    'jpy': [110, 120, 130]
}

data2 = {
    'last_updated': ['2024-06-07T00:00:00', '2024-06-09T00:00:00', '2024-06-09T00:00:00'],
    'instrument_type': ['cash', 'transfer', 'cash'],
    'bank': ['Bank A', 'Bank B', 'Bank C'],
    'deal_type': ['buy', 'sell', 'buy'],
    'jpy': [110, 120, 130],
    'usd': [1.1, 1.2, 1.3],
}

list_currency = ['USD', 'JPY']
exrate_pk = ['bank', 'deal_type', 'instrument_type']
select_columns = exrate_pk + list_currency
select_columns = [col.lower() for col in select_columns]


df1 = pd.DataFrame(data1)[select_columns]
df2 = pd.DataFrame(data2)[select_columns]

# Ensure the 'last_updated' column is in datetime format
# df1['last_updated'] = pd.to_datetime(df1['last_updated'])
# df2['last_updated'] = pd.to_datetime(df2['last_updated'])

# Sort DataFrames by the primary key fields
df1_sorted = df1.sort_values(by=['bank', 'deal_type', 'instrument_type'])
df2_sorted = df2.sort_values(by=['bank', 'deal_type', 'instrument_type'])

# Sort the columns alphabetically
df1_sorted = df1_sorted[sorted(df1_sorted.columns)]
df2_sorted = df2_sorted[sorted(df2_sorted.columns)]

# Set the primary key fields as the index
df1_indexed = df1_sorted.set_index(['bank', 'deal_type', 'instrument_type'])
df2_indexed = df2_sorted.set_index(['bank', 'deal_type', 'instrument_type'])

# Compare the DataFrames
are_equal = df1_indexed.equals(df2_indexed)

print("Are the DataFrames equal based on the primary key fields? ", are_equal)


Are the DataFrames equal based on the primary key fields?  True


In [46]:
select_columns

['bank', 'deal_type', 'instrument_type', 'usd', 'jpy']

In [36]:
df1

,bank,deal_type,instrument_type,last_updated,usd,jpy
0,Bank A,buy,cash,2024-06-07T00:00:00,1.1,110
1,Bank B,sell,transfer,2024-06-08T00:00:00,1.2,120
2,Bank C,buy,cash,2024-06-09T00:00:00,1.3,130


In [37]:
df2

,last_updated,instrument_type,bank,deal_type,jpy,usd
0,2024-06-07T00:00:00,cash,Bank A,buy,110,1.1
1,2024-06-08T00:00:00,transfer,Bank B,sell,120,1.2
2,2024-06-09T00:00:00,cash,Bank C,buy,130,1.3


In [1]:
import pandas as pd

data = {
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Catherine'],
    'age': [25, 30, 22],
    'city': ['New York', 'Los Angeles', 'Chicago']
}
df = pd.DataFrame(data)

# List of desired columns
desired_columns = ['id', 'name']
df = df.loc[df['city'] == 'New York', desired_columns]

In [7]:
import pandas as pd

# Sample DataFrame 1
data1 = {
    'USD': [25223.0],
    'AUD': [16445.54],
    'CAD': [18083.23],
    'CHF': [27587.38],
    'EUR': [26363.45],
    'JPY': [153.35],
    'SGD': [18272.37]
}
df1 = pd.DataFrame(data1)

# Sample DataFrame 2
data2 = {
    'USD': [25223.0],
    'AUD': [16445.54],
    'CAD': [18083.23],
    'CHF': [27587.38],
    'EUR': [26363.45],
    'JPY': [153.35],
    'SGD': [18272.37]
}
df2 = pd.DataFrame(data2)

# Compare the two DataFrames
are_equal = df1.equals(df2)

if not are_equal:
    print("Are the two DataFrames equal?", are_equal)
1

1

# Try PyFlink

## Ex 1 - Table API 

In [1]:
from pyflink.table import *
from pyflink.table.expressions import col

# environment configuration: main entry point for interacting with the Flink runtime.
t_env = TableEnvironment.create(environment_settings=EnvironmentSettings.in_batch_mode())

In [10]:
# registers a table named source and a table named sink in the table environment
source_data_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/source"
result_data_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/result"

source_ddl = f"""
        create table Orders(
            a VARCHAR,
            b BIGINT,
            c BIGINT,
            rowtime TIMESTAMP(3),
            WATERMARK FOR rowtime AS rowtime - INTERVAL '1' SECOND
        ) with (
            'connector' = 'filesystem',
            'format' = 'csv',
            'path' = '{source_data_path}'
        )
        """
t_env.execute_sql(source_ddl)

sink_ddl = f"""
    create table `Result`(
        a VARCHAR,
        cnt BIGINT
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = '{result_data_path}'
    )
    """
t_env.execute_sql(sink_ddl)

# specify table program
orders = t_env.from_path("Orders")  # schema (a, b, c, rowtime)

Py4JJavaError: An error occurred while calling o8.executeSql.
: org.apache.flink.table.api.ValidationException: Could not execute CreateTable in path `default_catalog`.`default_database`.`Orders`
	at org.apache.flink.table.catalog.CatalogManager.execute(CatalogManager.java:1298)
	at org.apache.flink.table.catalog.CatalogManager.createTable(CatalogManager.java:950)
	at org.apache.flink.table.operations.ddl.CreateTableOperation.execute(CreateTableOperation.java:86)
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.executeInternal(TableEnvironmentImpl.java:1107)
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.executeSql(TableEnvironmentImpl.java:735)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:104)
	at java.base/java.lang.reflect.Method.invoke(Method.java:577)
	at org.apache.flink.api.python.shaded.py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at org.apache.flink.api.python.shaded.py4

In [3]:
orders.group_by(col("a")).select(col("a"), col("b").count.alias('cnt')).execute_insert("Result").wait()


In [4]:
from pyflink.table.window import Tumble
from pyflink.table.expressions import col, lit

result = orders.filter(col("a").is_not_null & col("b").is_not_null & col("c").is_not_null) \
               .select(col("a").lower_case.alias('a'), col("b"), col("rowtime")) \
               .window(Tumble.over(lit(1).hour).on(col("rowtime")).alias("hourly_window")) \
               .group_by(col('hourly_window'), col('a')) \
               .select(col('a'), col('hourly_window').end.alias('hour'), col("b").avg.alias('avg_billing_amount'))

In [11]:
orders = t_env.from_path("Orders")
orders

## Ex 2

In [5]:
from pyflink.table import *
from pyflink.table.expressions import col

t_env = TableEnvironment.create(EnvironmentSettings.in_streaming_mode())
t_env.get_config().set("parallelism.default", "1")

In [6]:
input_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/source"
output_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/result"

my_source_ddl = """
    create table source (
        word STRING
    ) with (
        'connector' = 'filesystem',
        'format' = 'csv',
        'path' = '{}'
    )
""".format(input_path)

my_sink_ddl = """
    create table sink (
        word STRING,
        `count` BIGINT
    ) with (
        'connector' = 'filesystem',
        'format' = 'canal-json',
        'path' = '{}'
    )
""".format(output_path)

t_env.execute_sql(my_source_ddl)
t_env.execute_sql(my_sink_ddl)

In [ ]:
@udtf(result_types=[DataTypes.STRING()])
def split(line: Row):
    for s in line[0].split():
        yield Row(s)

# compute word count
tab.flat_map(split).alias('word') \
   .group_by(col('word')) \
   .select(col('word'), lit(1).count) \
   .execute_insert('sink') \
   .wait()

In [5]:
word_count_data = ["To be, or not to be,--that is the question:--",
                   "Whether 'tis nobler in the mind to suffer",
                   "The slings and arrows of outrageous fortune",
                   "Or to take arms against a sea of troubles,",
                   "And by opposing end them?--To die,--to sleep,--",
                   "No more; and by a sleep to say we end",
                   "The heartache, and the thousand natural shocks",
                   "That flesh is heir to,--'tis a consummation",
                   "Devoutly to be wish'd. To die,--to sleep;--",
                   "To sleep! perchance to dream:--ay, there's the rub;",
                   "For in that sleep of death what dreams may come,",
                   "When we have shuffled off this mortal coil,",
                   "Must give us pause: there's the respect",
                   "That makes calamity of so long life;",
                   "For who would bear the whips and scorns of time,",
                   "The oppressor's wrong, the proud man's contumely,",
                   "The pangs of despis'd love, the law's delay,",
                   "The insolence of office, and the spurns",
                   "That patient merit of the unworthy takes,",
                   "When he himself might his quietus make",
                   "With a bare bodkin? who would these fardels bear,",
                   "To grunt and sweat under a weary life,",
                   "But that the dread of something after death,--",
                   "The undiscover'd country, from whose bourn",
                   "No traveller returns,--puzzles the will,",
                   "And makes us rather bear those ills we have",
                   "Than fly to others that we know not of?",
                   "Thus conscience does make cowards of us all;",
                   "And thus the native hue of resolution",
                   "Is sicklied o'er with the pale cast of thought;",
                   "And enterprises of great pith and moment,",
                   "With this regard, their currents turn awry,",
                   "And lose the name of action.--Soft you now!",
                   "The fair Ophelia!--Nymph, in thy orisons",
                   "Be all my sins remember'd."]


In [9]:
from pyflink.table.udf import udtf

def word_count(input_path, output_path):
    t_env = TableEnvironment.create(EnvironmentSettings.in_streaming_mode())
    # write all the data to one file
    t_env.get_config().set("parallelism.default", "1")

    # define the source
    if input_path is not None:
        t_env.create_temporary_table(
            'source',
            TableDescriptor.for_connector('filesystem')
                .schema(Schema.new_builder()
                        .column('word', DataTypes.STRING())
                        .build())
                .option('path', input_path)
                .format('csv')
                .build())
        tab = t_env.from_path('source')
    else:
        print("Executing word_count example with default input data set.")
        print("Use --input to specify file input.")
        tab = t_env.from_elements(map(lambda i: (i,), word_count_data),
                                  DataTypes.ROW([DataTypes.FIELD('line', DataTypes.STRING())]))

    # define the sink
    if output_path is not None:
        t_env.create_temporary_table(
            'sink',
            TableDescriptor.for_connector('filesystem')
                .schema(Schema.new_builder()
                        .column('word', DataTypes.STRING())
                        .column('count', DataTypes.BIGINT())
                        .build())
                .option('path', output_path)
                .format(FormatDescriptor.for_format('canal-json')
                        .build())
                .build())
    else:
        print("Printing result to stdout. Use --output to specify output path.")
        t_env.create_temporary_table(
            'sink',
            TableDescriptor.for_connector('print')
                .schema(Schema.new_builder()
                        .column('word', DataTypes.STRING())
                        .column('count', DataTypes.BIGINT())
                        .build())
                .build())

    @udtf(result_types=[DataTypes.STRING()])
    def split(line: Row):
        for s in line[0].split():
            yield Row(s)

    # compute word count
    tab.flat_map(split).alias('word') \
        .group_by(col('word')) \
        .select(col('word'), lit(1).count) \
        .execute_insert('sink') \
        .wait()
    # remove .wait if submitting to a remote cluster, refer to
    # https://nightlies.apache.org/flink/flink-docs-stable/docs/dev/python/faq/#wait-for-jobs-to-finish-when-executing-jobs-in-mini-cluster
    # for more details

input_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/source"
output_path = "D:/DE/study_de/Data Engineering/ExchangeRateProject/test/result"
word_count(input_path, None)

Printing result to stdout. Use --output to specify output path.
